In [3]:
%matplotlib inline

Data reduction: apply pca analysis to reduce neuropsych test scores to latent component weights, for prediction of continuous variable (a latent memory score)

Questions: sparse (data missing for some participants: how to handle)?

scaling data (transform into z scores)



In [18]:
import numpy as np
import pandas as pd
import sklearn

from numpy import nan as NaN
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [19]:
# Step 1: load and clean up data

# Load neuropsych data as pandas dataframe
dset = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Participants/Neuropsych/ALL_Neuropsych_scores.tsv'
npsych = pd.read_csv(dset, sep = '\t')
# index data by subject id
npsych.set_index('dccid', inplace = True)
# convert all columns to float64
npsych = npsych.astype('float64', copy=False)
x = npsych.values
print('original data')
print(x)

# Filling up missing values (NaN) with multivariate imputer
# https://scikit-learn.org/stable/modules/impute.html
#imp_iter = IterativeImputer(max_iter=10, random_state=0)
#imp_iter.fit(x)
#IterativeImputer(add_indicator=False, estimator=None,
#                 imputation_order='ascending', initial_strategy='mean',
#                 max_iter=10, max_value=None, min_value=None,
#                 missing_values=NaN, n_nearest_features=None,
#                 random_state=0, sample_posterior=False, tol=0.001,
#                 verbose=0)
#x_test = np.round(imp_iter.transform(x))
#print(x_test)

imp_simple = SimpleImputer(missing_values=NaN, strategy='median')
imp_simple.fit(x)
SimpleImputer(copy=True, fill_value=None,
        missing_values=NaN, strategy='median', verbose=0)
x_full = imp_simple.transform(x)
print('inputed data')
print(x_full)

# Scaling data (normalize to mean = 0, sd = 1)
x_scaled = StandardScaler().fit_transform(x_full)
print('scaled data')
print(x_scaled)


original data
[[ 7.    8.   29.96 ...  0.   31.   16.  ]
 [ 7.    5.   55.68 ...  0.   28.   19.  ]
 [ 4.    2.   38.88 ...  0.5  28.   17.  ]
 ...
 [  nan   nan 29.72 ...  0.   27.   19.  ]
 [ 4.    5.   58.41 ...  0.5  29.   10.  ]
 [ 5.    6.   38.66 ...  0.   30.   14.  ]]
inputed data
[[ 7.    8.   29.96 ...  0.   31.   16.  ]
 [ 7.    5.   55.68 ...  0.   28.   19.  ]
 [ 4.    2.   38.88 ...  0.5  28.   17.  ]
 ...
 [ 5.    4.   29.72 ...  0.   27.   19.  ]
 [ 4.    5.   58.41 ...  0.5  29.   10.  ]
 [ 5.    6.   38.66 ...  0.   30.   14.  ]]
scaled data
[[ 0.91546815  1.5456917  -0.74462814 ... -0.50346504  0.84789328
   0.59459139]
 [ 0.91546815  0.2152774   0.97214333 ... -0.50346504 -0.53104895
   1.31509625]
 [-0.52779542 -1.11513691 -0.14923151 ...  1.41716087 -0.53104895
   0.83475968]
 ...
 [-0.04670756 -0.22819404 -0.76064778 ... -0.50346504 -0.99069636
   1.31509625]
 [-0.52779542  0.2152774   1.15436674 ...  1.41716087 -0.07140154
  -0.84641833]
 [-0.04670756  0.658748

In [26]:
# Step 2: run PCA

pca = PCA(n_components = 4)

components = pca.fit_transform(x_scaled)

weights = pd.DataFrame(data = components, 
                       columns = ['pc1', 'pc2', 'pc3', 'pc4'])

print(weights.shape, weights.dtypes, weights.head())

(103, 4) pc1    float64
pc2    float64
pc3    float64
pc4    float64
dtype: object         pc1       pc2       pc3       pc4
0 -2.241407  2.267159 -0.256705  1.628550
1 -0.955390 -1.662160 -0.949029 -0.725271
2  1.724027  0.995476 -1.177703  0.241944
3 -0.667614  1.529444 -0.010563  0.746561
4  3.489571 -1.773405 -1.319734  0.293359


In [34]:
print(pca.explained_variance_ratio_)
print(pca.components_.shape)
print(pca.components_[0])

factors = pca.components_[0].transpose()
print(factors.shape)
print(factors)

[0.24841829 0.09074857 0.08344591 0.06869135]
(4, 21)
[-0.25939161 -0.30553407  0.11582782  0.25601196  0.0627154  -0.21301552
  0.03014876  0.05744948 -0.35112199 -0.34253406 -0.19738565 -0.22433763
 -0.13785103 -0.14637503 -0.20489357 -0.16556378 -0.23445007 -0.08720294
  0.30902324 -0.22372413 -0.24298859]
(21,)
[-0.25939161 -0.30553407  0.11582782  0.25601196  0.0627154  -0.21301552
  0.03014876  0.05744948 -0.35112199 -0.34253406 -0.19738565 -0.22433763
 -0.13785103 -0.14637503 -0.20489357 -0.16556378 -0.23445007 -0.08720294
  0.30902324 -0.22372413 -0.24298859]
